In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import geohash

In [2]:
import os
os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'

In [3]:
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
#os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
#os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/usr/local/opt/apache-spark/libexec


In [4]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark'))
sc = pyspark.SparkContext(conf=conf)

In [5]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)

In [5]:
g_rdd = sc.textFile('green_tripdata_2013-10.csv,green_tripdata_2015-01.csv')
g_rdd = g_rdd.map(lambda line: tuple(line.split(',')))
g_rdd.take(2)

[(u'2',
  u'2013-10-01 00:00:00',
  u'2013-10-01 15:33:36',
  u'N',
  u'1',
  u'0',
  u'0',
  u'-73.903465270996094',
  u'40.845088958740234',
  u'1',
  u'.19',
  u'42',
  u'0',
  u'0.5',
  u'0',
  u'0',
  u'',
  u'42.5',
  u'2',
  u'1',
  u'',
  u''),
 (u'2',
  u'2013-10-01 00:00:00',
  u'2013-10-01 09:01:53',
  u'N',
  u'1',
  u'0',
  u'0',
  u'-73.937408447265625',
  u'40.758129119873047',
  u'1',
  u'.00',
  u'2.5',
  u'0',
  u'0.5',
  u'0',
  u'0',
  u'',
  u'3',
  u'2',
  u'1',
  u'',
  u'')]

In [12]:
y_rdd = sc.textFile('yellow_tripdata_2015-02.csv')
y_rdd = y_rdd.map(lambda line: tuple(line.split(',')))
y_rdd.take(2)

[(u'2',
  u'2015-02-08 11:33:46',
  u'2015-02-08 11:37:45',
  u'1',
  u'.56',
  u'-73.949905395507813',
  u'40.717437744140625',
  u'1',
  u'N',
  u'-73.95001220703125',
  u'40.724010467529297',
  u'1',
  u'4.5',
  u'0',
  u'0.5',
  u'1.06',
  u'0',
  u'0.3',
  u'6.36'),
 (u'2',
  u'2015-02-28 23:52:39',
  u'2015-03-01 00:00:03',
  u'1',
  u'1.24',
  u'-73.983497619628906',
  u'40.756103515625',
  u'1',
  u'N',
  u'-73.989540100097656',
  u'40.767860412597656',
  u'1',
  u'6.5',
  u'0.5',
  u'0.5',
  u'1.56',
  u'0',
  u'0.3',
  u'9.36')]

### Data preparation specification
Given, a certain granularity in location (geohash length g), granularity in time (bins per day b) and a chosen wideness (w) of the neighbourhood we want to look at, the aggregated data in the end should have the following columns

#### "geohash"
Geohash with length g (categorical feature). This column will not actually be used in the prediction. It is just an id and can be used when calculating the distances between the geohashes.
#### "time_cat" 
Time of the day as a categorical feature. If b = 24 (one bin for every hour), then "time_cat" for a pickup at 14:20:00 should be the string "14:00". If b = 96 (one bin for every quarter of an hour), then "time_cat" for a pickup at 14:20:00 should be the string "14:15".
#### "time_num" 
Time of the day as a (binned!) floating point number between 0 and 1, where the center of the bin is converted to a floating point number between 0 and 1. So if b = 24, then "time_num" for a pickup at 14:20:00 should be 14.5 / 24 =  0.6042. If b = 96, it should translate to 14.375 / 24 = 0.5990.
#### "time_cos" 
The binned "time_num" variable converted to a cosine version so that time nicely 'loops' rather than going saw-like when it traverses midnight. I'm not sure if it adds much predictive power, but only one way to find out. "time_cos" = cos(time_num * 2 * Pi). So for 24 bins, 14:20:00 would translate to cos(0.6042 * 2 * Pi) = -0.7932.
#### "time_sin" 
Same thing as 4) but then with sine. So, "time_sin" = sin(time_num * 2 * Pi). For 24 bins per day, 14:20:00 would translate to sin(0.6042 * 2 * Pi) = -0.6089.
#### "day_cat" 
Day of the week as a categorical feature: "Monday", "Tuesday", etc.
#### "day_num" 
Day of the week as  a numerical feature going from 0 (Monday morning, start of the week) to 1 (Sunday night). Yeah, it's European to start the week on Monday. Whatever, haha :P. Anyway, with 24 bins, Tuesday afternoon 14:20:00 would translate to (1 + 14.5/24)/7 = 0.2292.
#### "day_cos" 
Binned "day_num" variable converted to a cosine version. "day_cos" = cos(day_num * 2 * Pi)
#### "day_sin" 
Binned "day_num"variable converted to a sine version. "day_sin" = sin(day_num * 2 * Pi)
#### "weekend" 
0 if weekday, 1 if weekend (Saturday/Sunday)
#### "holiday" (not included yet)
0 if normal day, 1 if holiday. Not sure if it's easy to find all the holidays in NYC from 2013 - 2015…
to xxxxx) 
#### Location features 
For each binned location, there should be a feature. The name of each feature should be the geohash. To calculate the value of the feature, you need the distance between the location of the row and the location of the column. The value should be calculated as 1/(distance_between_locations_in_km + 1)^w (i.e. inverse distance weighting). The larger w gets, the quicker everything drops to 0. So, for w = 4, the value for the location itself will be 1/(0 + 1)^4 = 1. For a location 2 km away, it will be 1/(2 + 1)^4 = 1/3^4 = 0.012. So information about pickup rates from locations 2 km away are pretty much not taken into account at all when predicting for a certain location. If w = 2, the value for a location 2 km away is a bit larger: 0.11. If w = 1, it is even larger: 0.33. So w is a parameter with which we can determine how 'smooth' are predictions are going to be. The larger we take w, the more we look at the neighbourhoods, the smoother the predictions will be. What the optimal w is, I don't know. I think we should try a few different options and see which works best on the validation set.

In the end, this should result in a number of records equal to 7 * b * x where x is the number of locations. For b = 24 and a geohash length g of 7, this should come down to about 24 * 7 * 10000 = roughly 1 to 2 million records.


In [11]:
import time
from datetime import date
import math
def date_extractor(date_str,b,minutes_per_bin):
    # Takes a datetime object as a parameter
    # and extracts and returns a tuple of the form: (as per the data specification)
    # (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)
    # Split date string into list of date, time
    
    d = date_str.split()
    # TIME
    
    #list of hour,min,sec
    time_list = [int(t) for t in d[1].split(':')]
    # calculate number of minute into the day
    num_minutes = time_list[0] * 60 + time_list[1]
    
    # round
    minutes_over_prev_bin = num_minutes % minutes_per_bin
    time_bin = num_minutes / minutes_per_bin
    if minutes_over_prev_bin >= (minutes_per_bin / 2.0):
        time_bin += minutes_per_bin
    
    hour_bin = time_bin / 60
    min_bin = time_bin % 60
    #get time_cat
    hour_str = str(hour_bin) if hour_bin / 10 > 0 else "0" + str(hour_bin)
    min_str = str(min_bin) if min_bin / 10 > 0 else "0" + str(min_bin)
    time_cat = hour_str + ":" + min_str
    
    # Get a floating point representation of the center of the time bin
    time_num = float(hour_bin + (min_bin+(minutes_per_bin / 2.0))/60.0)
    
    time_cos = math.cos(time_num * 2 * math.pi)
    time_sin = math.sin(time_num * 2 * math.pi)
    
    # DATE
    # Parse year, month, day
    date_list = d[0].split('-')
    d_obj = date(int(date_list[0]),int(date_list[1]),int(date_list[2]))
    day_to_str = {0: "Monday",
                  1: "Tuesday",
                  2: "Wednesday",
                  3: "Thursday",
                  4: "Friday",
                  5: "Saturday",
                  6: "Sunday"}
    day_of_week = d_obj.weekday()
    day_cat = day_to_str[day_of_week]
    day_num = (day_of_week + time_num/24)/7.0
    day_cos = math.cos(day_num * 2 * math.pi)
    day_sin = math.sin(day_num * 2 * math.pi)
    
    weekend = 0
    #check if it is the weekend
    if day_of_week in [5,6]:
        weekend = 1
    
    
    return (time_cat, time_num, time_cos, time_sin, day_cat, day_num, day_cos, day_sin, weekend)

import geohash
def data_cleaner(zipped_row):
    # takes a tuple (row,g,b,minutes_per_bin) as a parameter and returns a tuple of the form:
    # (day of the week, hour, geotag)
    row = zipped_row[0]
    g = zipped_row[1]
    b = zipped_row[2]
    minutes_per_bin = zipped_row[3]
    # The indices of pickup datetime, longitude, and latitude respectively
    indices = (1, 6, 5)
    
    #extract day of the week and hour
    date_str = row[indices[0]]
    clean_date = date_extractor(date_str,b,minutes_per_bin)
    #assert len(clean_date) == 2, "clean date is len %r" % len(clean_date)
    #get geo hash

    latitude = float(row[indices[1]])
    longitude = float(row[indices[2]])
    location = None
    if latitude < 50 and latitude > 35 and longitude < -50 and longitude > -80:
        location = geohash.encode(latitude,longitude, g)
    #else:
    #    print "Invalid latitude ("+str(latitude)+") or longitude("+str(longitude)+")"
    #if latitude == 0 or longitude == 0:
    #    location = '0'*g
    #else:
    #    location = geohash.encode(latitude,longitude, g)
    # I was having trouble importing geohash on AWS, so for now we round to 3 decimal places to discretize the data
    # At New York's location,
    # .001 degree latitude is approximately 100 meters
    # .001 degree longitude is less than 100 meters
    #location = (round(latitude,3), round(longitude,3))

    return tuple(list(clean_date)+[location])

#### Specify Parameters

In [7]:
# Clean Data
# Create Data as Specified
# Parameters
g = 7 #geohash length
b = 24 # number of time bins per day
# Note: b must evenly divide 60
minutes_per_bin = int((24 / float(b)) * 60)


In [8]:
gclean_rdd = g_rdd.map(lambda row: (row, g, b, minutes_per_bin))\
                .map(data_cleaner)\
                .map(lambda row: (row,1))\
                .reduceByKey(lambda a,b: a + b)

In [9]:
gclean_rdd.count()

555424

In [10]:
gclean_rdd.take(50)

[(('01:20',
   1.8333333333333335,
   0.5000000000000007,
   -0.8660254037844383,
   'Monday',
   0.010912698412698414,
   0.9976502379123289,
   0.06851279291835671,
   0,
   'dr5rvv0'),
  1),
 (('00:20',
   0.8333333333333334,
   0.5,
   -0.8660254037844386,
   'Thursday',
   0.43353174603174605,
   -0.914051811437943,
   0.4055974432944752,
   0,
   'dr5ry2b'),
  1),
 (('01:14',
   1.7333333333333334,
   -0.10452846326765272,
   -0.9945218953682734,
   'Wednesday',
   0.29603174603174603,
   -0.2852104494978021,
   0.9584649182402357,
   0,
   'dr72mt7'),
  1),
 (('00:10',
   0.6666666666666666,
   -0.5000000000000004,
   -0.8660254037844384,
   'Tuesday',
   0.1468253968253968,
   0.6038044103254775,
   0.7971325072229223,
   0,
   'dr72mb8'),
  2),
 (('00:23',
   0.8833333333333333,
   0.7431448254773937,
   -0.6691306063588588,
   'Friday',
   0.5766865079365079,
   -0.886145814907491,
   -0.4634065113072309,
   0,
   'dr5rwzu'),
  4),
 (('00:09',
   0.65,
   -0.5877852522924732,

In [13]:
yclean_rdd = y_rdd.map(lambda row: (row, g, b, minutes_per_bin))\
                .map(data_cleaner)\
                .map(lambda row: (row,1))\
                .reduceByKey(lambda a,b: a + b)

In [14]:
yclean_rdd.count()

859235

In [15]:
yclean_rdd.take(5)

[(('00:10',
   0.6666666666666666,
   -0.5000000000000004,
   -0.8660254037844384,
   'Wednesday',
   0.28968253968253965,
   -0.24675739769029345,
   0.969077286229078,
   0,
   'dr5rumv'),
  35),
 (('01:19',
   1.8166666666666667,
   0.4067366430758003,
   -0.9135454576426009,
   'Saturday',
   0.7250992063492063,
   -0.15581877706340935,
   -0.9877856592978376,
   1,
   'dr5rkhf'),
  3),
 (('00:05',
   0.5833333333333334,
   -0.8660254037844386,
   -0.5000000000000001,
   'Tuesday',
   0.1463293650793651,
   0.6062858642634977,
   0.7952467860948975,
   0,
   'dr5revn'),
  1),
 (('00:19',
   0.8166666666666667,
   0.40673664307579976,
   -0.9135454576426011,
   'Monday',
   0.00486111111111111,
   0.9995335908367129,
   0.030538513209822652,
   0,
   'dr5rvth'),
  3),
 (('00:22',
   0.8666666666666667,
   0.6691306063588585,
   -0.743144825477394,
   'Friday',
   0.5765873015873016,
   -0.8864344987859896,
   -0.46285405838345145,
   0,
   'dr5rkw8'),
  2)]